# Regulary fetch latest batch of features from feature store
# Current model in model registry

# inference pipeline - will compute new predicctions based on this data and 
# save it back to the feature store (predictions?)

In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.Timestamp.now(tz='UTC').floor('h')
current_date =  current_date - timedelta(days=180)

#current_date = pd.to_datetime('2024-10-30 23:00:00')
#current_date = pd.to_datetime('2024-10-02 23:00:00')
print(f'{current_date=}')

current_date=Timestamp('2024-08-16 00:00:00+0000', tz='UTC')


In [25]:
current_date

Timestamp('2024-08-16 00:00:00+0000', tz='UTC')

In [ ]:
# TESTING RETURNING THE PICKUP_TS column
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

2025-02-12 00:06:50,548 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-12 00:06:50,552 INFO: Initializing external client
2025-02-12 00:06:50,553 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-12 00:06:51,653 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192104
2025-02-12 00:06:54,060 INFO: Feature view already exists, skipping creation.
2025-02-12 00:06:54,062 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-12 00:06:54,070 INFO: Initializing external client
2025-02-12 00:06:54,070 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-12 00:06:55,191 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192104
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (11.48s) 
Index(['pickup_hour', 'rides', 'pickup_location_id', 'pickup_ts'], dtype='object')
** More data

In [27]:
features.columns

Index(['rides_previous_672_hour', 'rides_previous_671_hour',
       'rides_previous_670_hour', 'rides_previous_669_hour',
       'rides_previous_668_hour', 'rides_previous_667_hour',
       'rides_previous_666_hour', 'rides_previous_665_hour',
       'rides_previous_664_hour', 'rides_previous_663_hour',
       ...
       'rides_previous_8_hour', 'rides_previous_7_hour',
       'rides_previous_6_hour', 'rides_previous_5_hour',
       'rides_previous_4_hour', 'rides_previous_3_hour',
       'rides_previous_2_hour', 'rides_previous_1_hour', 'pickup_hour',
       'pickup_location_id'],
      dtype='object', length=674)

In [ ]:
features

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id
0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,...,1.0,1.0,3.0,0.0,1.0,0.0,0.0,1.0,2024-08-16 00:00:00+00:00,1
1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2024-08-16 00:00:00+00:00,3
2,16.0,10.0,5.0,2.0,1.0,0.0,5.0,9.0,10.0,6.0,...,4.0,5.0,4.0,1.0,3.0,3.0,9.0,5.0,2024-08-16 00:00:00+00:00,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-08-16 00:00:00+00:00,5
4,1.0,0.0,2.0,1.0,1.0,1.0,5.0,1.0,5.0,4.0,...,3.0,1.0,4.0,1.0,2.0,1.0,3.0,1.0,2024-08-16 00:00:00+00:00,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,12.0,6.0,3.0,2.0,3.0,4.0,3.0,11.0,5.0,17.0,...,49.0,60.0,61.0,53.0,55.0,23.0,26.0,20.0,2024-08-16 00:00:00+00:00,261
192,9.0,3.0,1.0,0.0,3.0,24.0,63.0,109.0,149.0,97.0,...,64.0,72.0,75.0,54.0,36.0,24.0,15.0,9.0,2024-08-16 00:00:00+00:00,262
193,25.0,25.0,15.0,6.0,16.0,29.0,68.0,85.0,95.0,128.0,...,104.0,104.0,132.0,86.0,95.0,97.0,88.0,42.0,2024-08-16 00:00:00+00:00,263
194,14.0,8.0,4.0,1.0,5.0,4.0,7.0,7.0,14.0,10.0,...,23.0,14.0,23.0,29.0,17.0,17.0,14.0,13.0,2024-08-16 00:00:00+00:00,264


In [29]:
# # Make a copy of original dataframe and store pickup_ts separately
# original_features = features_with_pickup_ts_column.copy()
# pickup_ts_series = features_with_pickup_ts_column['pickup_ts'].copy()


# # Remove pickup_ts from features for predictions
# features_for_prediction = features_with_pickup_ts_column.drop('pickup_ts', axis = 1 )
# features_for_prediction

## Be aware that here we are making predictions without the 'pickup_ts' column 

In [30]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

2025-02-12 00:07:10,932 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-12 00:07:10,934 INFO: Initializing external client
2025-02-12 00:07:10,935 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-12 00:07:11,902 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192104
[LightGBM] [Warning] feature_fraction is set=0.48506182283482746, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.48506182283482746
[LightGBM] [Warning] bagging_fraction is set=0.3883490522365708, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3883490522365708


In [31]:
predictions

,pickup_location_id,predicted_demand
0,1,21.0
1,3,23.0
2,4,24.0
3,5,21.0
4,7,24.0
...,...,...
191,261,21.0
192,262,22.0
193,263,22.0
194,264,22.0


In [32]:
# pickup_hour = features_with_pickup_ts_column['pickup_hour'][0]
# pickup_hour

In [33]:
from src.inference import read_incremented_date

predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,21.0,2024-08-16 00:00:00+00:00
1,3,23.0,2024-08-16 00:00:00+00:00
2,4,24.0,2024-08-16 00:00:00+00:00
3,5,21.0,2024-08-16 00:00:00+00:00
4,7,24.0,2024-08-16 00:00:00+00:00
...,...,...,...
191,261,21.0,2024-08-16 00:00:00+00:00
192,262,22.0,2024-08-16 00:00:00+00:00
193,263,22.0,2024-08-16 00:00:00+00:00
194,264,22.0,2024-08-16 00:00:00+00:00


## Add pickup_ts back into dataframe 

In [ ]:
predictions['pickup_ts'] = predictions['pickup_hour'].astype(int) // 10**6
predictions

,pickup_location_id,predicted_demand,pickup_hour,pickup_ts
0,1,21.0,2024-08-16 00:00:00+00:00,1723766400
1,3,23.0,2024-08-16 00:00:00+00:00,1723766400
2,4,24.0,2024-08-16 00:00:00+00:00,1723766400
3,5,21.0,2024-08-16 00:00:00+00:00,1723766400
4,7,24.0,2024-08-16 00:00:00+00:00,1723766400
...,...,...,...,...
191,261,21.0,2024-08-16 00:00:00+00:00,1723766400
192,262,22.0,2024-08-16 00:00:00+00:00,1723766400
193,263,22.0,2024-08-16 00:00:00+00:00,1723766400
194,264,22.0,2024-08-16 00:00:00+00:00,1723766400


In [45]:
# 2 ways to try this: 

# (1) -  you have put in both pickup_hour and pickup_ts in here 
# and Pau's only has pickup hour. Try with this and then if not, try with pickup_ts
# (2) - 

from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS, # model_predictions_feature_group
    version=4,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_ts'],
    event_time='pickup_ts',
)

2025-02-12 00:14:58,564 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-12 00:14:58,570 INFO: Initializing external client
2025-02-12 00:14:58,570 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-12 00:14:59,583 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192104


In [46]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1192104/fs/1181777/fg/1398874


Uploading Dataframe: 100.00% |██████████| Rows 196/196 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: model_predictions_feature_group_4_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1192104/jobs/named/model_predictions_feature_group_4_offline_fg_materialization/executions


(Job('model_predictions_feature_group_4_offline_fg_materialization', 'SPARK'),
 None)